# LLM Optimization Modelling Experiment

In [156]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [373]:
problem = '''You are the person in charge of packing in a large company. Your job is to skillfully pack items of various weights in a box with predetermined capacity. The objective is to use as few boxes as possible. There is a set of items and a set of boxes with an upper bound on the amount of boxes. Each of the items has a known weight. Each item is packed into one box and items are packed into the chosen boxes. Please formulate a mathematical optimization model for this problem. '''

## 2. Generate the mathematical model

In [454]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [455]:
#Show the resopnse in a formatted way
Markdown(response.text)

Let's think step by step to formulate a mathematical optimization model for this problem.

**Sets:**

*  $I$: Set of items 
* $J$: Set of boxes 

**Parameters:**

* $w_i$: Weight of item $i \in I$
* $C$: Capacity of each box
* $N$: Maximum number of boxes allowed

**Decision Variables:**
* $x_{ij}$: Binary variable equal to 1 if item  $i \in I$ is packed in box $j \in J$, and 0 otherwise. 
* $y_j$: Binary variable equal to 1 if box $j \in J$ is used, and 0 otherwise.

**Mathematical Model:**

Minimize $\sum_{j \in J} y_j$

Subject to:
1.  $\sum_{j \in J} x_{ij} = 1,  \forall i \in I$  (Each item is assigned to exactly one box).

2.  $\sum_{i \in I} w_i x_{ij} \leq Cy_j, \forall j \in J$ (The total weight of items packed in a box cannot exceed its capacity).

3. $\sum_{j \in J} y_j \leq N$ (The total number of boxes used cannot exceed the maximum allowed).

4. $x_{ij} \in \{0,1\}, \forall i \in I, \forall j \in J$

5. $y_j \in \{0,1\}, \forall j \in J$ 

This model aims to minimize the total number of boxes used while ensuring that all items are packed, the capacity of each box is not exceeded, and the maximum number of boxes allowed is not surpassed. 


## 3. Generate the pyomo code

In [456]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [457]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data
items = ['A', 'B', 'C', 'D', 'E']
weights = {'A': 10, 'B': 15, 'C': 8, 'D': 5, 'E': 12}  # Sample weights
boxes = ['Box1', 'Box2', 'Box3']  # Sample boxes
capacity = 25  # Sample capacity for each box
max_boxes = 3  # Sample maximum number of boxes

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=items)
model.J = pyo.Set(initialize=boxes)

# Parameters
model.w = pyo.Param(model.I, initialize=weights)
model.C = pyo.Param(initialize=capacity)
model.N = pyo.Param(initialize=max_boxes)

# Decision Variables
model.x = pyo.Var(model.I, model.J, within=pyo.Binary)
model.y = pyo.Var(model.J, within=pyo.Binary)

# Objective Function
def objective_rule(model):
    return sum(model.y[j] for j in model.J)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
def constraint1_rule(model, i):
    return sum(model.x[i,j] for j in model.J) == 1
model.constraint1 = pyo.Constraint(model.I, rule=constraint1_rule)

def constraint2_rule(model, j):
    return sum(model.w[i] * model.x[i,j] for i in model.I) <= model.C * model.y[j]
model.constraint2 = pyo.Constraint(model.J, rule=constraint2_rule)

def constraint3_rule(model):
    return sum(model.y[j] for j in model.J) <= model.N
model.constraint3 = pyo.Constraint(rule=constraint3_rule)

# Solve
solver = pyo.SolverFactory('glpk')  # You can choose a different solver if desired
solver.solve(model)

# Print results
print("Optimal Solution:")
for j in model.J:
    if pyo.value(model.y[j]) == 1:
        print(f"Box {j} is used.")
        for i in model.I:
            if pyo.value(model.x[i,j]) == 1:
                print(f"Item {i} is packed in Box {j}.")
```

This code first defines sample data for the items, weights, boxes, capacity, and maximum number of boxes. Then, it creates a Pyomo model with sets, parameters, decision variables, objective function, and constraints as defined in the mathematical formulation. Finally, it calls a solver to find the optimal solution and prints the results.

Remember that the sample data is only for illustration purposes. You should replace it with your actual data.


## 4. Input problem data and try running the generated code

In [459]:
import pyomo.environ as pyo

# Sample Data
items = range(1,25)
weights = {1: 2,2: 2,3: 2,4: 2,5: 3,6: 3,7:4,8: 4,9: 4,10: 4,11: 4,12:4,13: 5,14:5,15: 5,16: 5,17: 5,18: 5,19: 6,20:6,21: 7,22: 7,23: 8, 24:8} # Weight of each item
capacity = 9
boxes = range(1,14)  # Sample boxes
capacity = 9  # Sample capacity for each box
max_boxes = 13  # Sample maximum number of boxes

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=items)
model.J = pyo.Set(initialize=boxes)

# Parameters
model.w = pyo.Param(model.I, initialize=weights)
model.C = pyo.Param(initialize=capacity)
model.N = pyo.Param(initialize=max_boxes)

# Decision Variables
model.x = pyo.Var(model.I, model.J, within=pyo.Binary)
model.y = pyo.Var(model.J, within=pyo.Binary)

# Objective Function
def objective_rule(model):
    return sum(model.y[j] for j in model.J)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
def constraint1_rule(model, i):
    return sum(model.x[i,j] for j in model.J) == 1
model.constraint1 = pyo.Constraint(model.I, rule=constraint1_rule)

def constraint2_rule(model, j):
    return sum(model.w[i] * model.x[i,j] for i in model.I) <= model.C * model.y[j]
model.constraint2 = pyo.Constraint(model.J, rule=constraint2_rule)

def constraint3_rule(model):
    return sum(model.y[j] for j in model.J) <= model.N
model.constraint3 = pyo.Constraint(rule=constraint3_rule)

# Solve
solver = pyo.SolverFactory('glpk')  # You can choose a different solver if desired
solver.solve(model)

# Print results
print("Optimal Solution:")
for j in model.J:
    if pyo.value(model.y[j]) == 1:
        print(f"Box {j} is used.")
        for i in model.I:
            if pyo.value(model.x[i,j]) == 1:
                print(f"Item {i} is packed in Box {j}.")

Optimal Solution:
Box 1 is used.
Item 24 is packed in Box 1.
Box 2 is used.
Item 1 is packed in Box 2.
Item 21 is packed in Box 2.
Box 3 is used.
Item 22 is packed in Box 3.
Box 4 is used.
Item 7 is packed in Box 4.
Item 18 is packed in Box 4.
Box 5 is used.
Item 20 is packed in Box 5.
Box 6 is used.
Item 8 is packed in Box 6.
Item 14 is packed in Box 6.
Box 7 is used.
Item 10 is packed in Box 7.
Item 13 is packed in Box 7.
Box 8 is used.
Item 9 is packed in Box 8.
Item 16 is packed in Box 8.
Box 9 is used.
Item 5 is packed in Box 9.
Item 19 is packed in Box 9.
Box 10 is used.
Item 11 is packed in Box 10.
Item 17 is packed in Box 10.
Box 11 is used.
Item 12 is packed in Box 11.
Item 15 is packed in Box 11.
Box 12 is used.
Item 23 is packed in Box 12.
Box 13 is used.
Item 2 is packed in Box 13.
Item 3 is packed in Box 13.
Item 4 is packed in Box 13.
Item 6 is packed in Box 13.


In [460]:
model.obj()

13.0

In [439]:
#Making the data for the problem
def BinPackingExample():
    B = 9
    w = [2,3,4,5,6,7,8]
    q = [4,2,6,6,2,2,2]
    s=[]
    for j in range(len(w)):
        for i in range(q[j]):
            s.append(w[j])
    return s,B

#print(BinPackingExample())
def FFD(s, B):
    remain = [B]
    sol = [[]]
    for item in sorted(s, reverse=True):
        for j,free in enumerate(remain):
            if free >= item:
                remain[j] -= item
                sol[j].append(item)
                break
        else:
            sol.append([item])
            remain.append(B-item)
    return sol
s,B = BinPackingExample()
n = len(s)
U = len(FFD(s, B))
print(s) #Weights
print(B) #Max bin capacity
print(n) #Number of items
print(U) #Max number of boxes

[2, 2, 2, 2, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 7, 7, 8, 8]
9
24
13


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [463]:
print(response.text)

Let's think step by step to formulate a mathematical optimization model for this problem.

**Sets:**

*  $I$: Set of items 
* $J$: Set of boxes 

**Parameters:**

* $w_i$: Weight of item $i \in I$
* $C$: Capacity of each box
* $N$: Maximum number of boxes allowed

**Decision Variables:**
* $x_{ij}$: Binary variable equal to 1 if item  $i \in I$ is packed in box $j \in J$, and 0 otherwise. 
* $y_j$: Binary variable equal to 1 if box $j \in J$ is used, and 0 otherwise.

**Mathematical Model:**

Minimize $\sum_{j \in J} y_j$

Subject to:
1.  $\sum_{j \in J} x_{ij} = 1,  \forall i \in I$  (Each item is assigned to exactly one box).

2.  $\sum_{i \in I} w_i x_{ij} \leq Cy_j, \forall j \in J$ (The total weight of items packed in a box cannot exceed its capacity).

3. $\sum_{j \in J} y_j \leq N$ (The total number of boxes used cannot exceed the maximum allowed).

4. $x_{ij} \in \{0,1\}, \forall i \in I, \forall j \in J$

5. $y_j \in \{0,1\}, \forall j \in J$ 

This model aims to minimize the 

In [464]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data
items = ['A', 'B', 'C', 'D', 'E']
weights = {'A': 10, 'B': 15, 'C': 8, 'D': 5, 'E': 12}  # Sample weights
boxes = ['Box1', 'Box2', 'Box3']  # Sample boxes
capacity = 25  # Sample capacity for each box
max_boxes = 3  # Sample maximum number of boxes

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=items)
model.J = pyo.Set(initialize=boxes)

# Parameters
model.w = pyo.Param(model.I, initialize=weights)
model.C = pyo.Param(initialize=capacity)
model.N = pyo.Param(initialize=max_boxes)

# Decision Variables
model.x = pyo.Var(model.I, model.J, within=pyo.Binary)
model.y = pyo.Var(model.J, within=pyo.Binary)

# Objective Function
def objective_rule(model):
    return sum(model.y[j] for j in model.J)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
def constraint1_rule(model, i):
    return sum(model.x[i,j] for j in model.J) == 1
model.constraint1 = pyo.Constraint(model.I, rule=const